In [1]:
! pip install semantic-kernel -U

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


In [3]:
from semantic_kernel.connectors.ai.open_ai.semantic_functions.open_ai_chat_prompt_template import (
    OpenAIChatPromptTemplate,
)
from semantic_kernel.connectors.ai.open_ai.utils import (
    chat_completion_with_function_call,
    get_function_calling_object,
)

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
kernel.add_chat_service("azure_chat_competion_service", AzureChatCompletion(deployment,endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_hotels",
            "description": "Retrieves hotels from the search index based on the parameters provided",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location of the hotel (i.e. Seattle, WA)"
                    },
                    "max_price": {
                        "type": "number",
                        "description": "The maximum price for the hotel"
                    },
                    "features": {
                        "type": "string",
                        "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                    }
                },
                "required": ["location"]
            }
        },
    }
]

In [7]:
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    functions=tools,
    function_call="auto",
)

In [8]:
prompt_template = OpenAIChatPromptTemplate(
    "{{$user_input}}", kernel.prompt_template_engine, prompt_config
)
prompt_template.add_user_message("Hi there, who are you?")
prompt_template.add_assistant_message(
    "I am Mosscap, a chat bot. I'm trying to figure out what people need."
)

In [9]:
function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)

In [10]:
chat_function = kernel.register_semantic_function("ChatBot", "Chat", function_config)

The function_call and functions parameters are deprecated. Please use the tool_choice and tools parameters instead.
The function_call and functions parameters are deprecated. Please use the tool_choice and tools parameters instead.


In [11]:
context = kernel.create_new_context()
context.variables[
        "user_input"
    ] = "I want to find a hotel in Seattle with free wifi and a pool."

In [12]:
filter = {"exclude_skill": ["ChatBot"]}
functions = get_function_calling_object(kernel, filter)

In [13]:
context = await chat_completion_with_function_call(
        kernel,
        chat_skill_name="ChatBot",
        chat_function_name="Chat",
        context=context
)

In [14]:
context

SKContext(memory=NullMemory(), variables=ContextVariables(variables={'input': '', 'user_input': 'I want to find a hotel in Seattle with free wifi and a pool.'}), skill_collection=ReadOnlySkillCollection(data={'chatbot': {'chat': SKFunction()}}))

In [15]:
context.result

''